In [1]:
! pip install -qU openai pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.1/755.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 28.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.


In [1]:
import pandas as pd
import re, json
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed

/Users/zakaria/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# import os, getpass

# os.environ['OPENAI_API_KEY'] = getpass.getpass("🔑 Enter your OpenAI API key: ")
from dotenv import load_dotenv
load_dotenv()

True

## Get list skills from provided profile Tools

## Code

In [3]:
SYS_TMPL_EXTRACT_SKILLS = """
You are a professional skill extraction engine.

#TASK#
Extract relevant professional skills from the input text, which may include job titles, structured metadata, descriptions, or any combination.

#REQUIREMENTS#
- Only output a clean, comma-separated list of skills.
- Do not include any extra text, labels, explanations, quotes, or commentary.
- Do not include general phrases like "or other relevant languages".
- Do not include placeholders or assumptions as text.
- Return only real, specific skills.
- If no valid skills are found, return exactly: null

#VALID OUTPUT EXAMPLES#
✅ Correct: Python, Kubernetes, cloud infrastructure, team leadership
❌ Wrong: Extracted Skills: Python, Kubernetes...
❌ Wrong: Or other skills like communication...
❌ Wrong: "null" — just write: null

Your output must follow this format **exactly**.
"""

USER_TMPL_EXTRACT_SKILLS = """{experience_input}"""

SKILL_CATEGORIES = [
    # Core Marketing
    "SEO", "Social Media", "Email Marketing", "Digital Advertising",
    "Influencer Marketing", "Affiliate Marketing", "Marketing Automation",
    "Content Marketing", "Copywriting", "Creative", "Design", "Strategy", "Brand",
    "Product Marketing", "Marketing Management", "Campaign Management",
    "Lead Generation", "Sales Management", "Growth Marketing", "CRM",
    "Data & Analytics", "Market Research", "Customer Experience", "Events",
    "Public Relations & Communications", "Direct Mail", "Channel Marketing",
    "Trade Marketing", "Marketing Operations", "Media Management",
    "Lifecycle Marketing", "Advertising",

    # Technology & Tools
    "Software/Tools", "Technical/Coding", "Equipment/Hardware",

    # Skills & Competencies
    "Soft Skills", "General Management", "Personal Attributes",

    # Professional Context
    "Job Functions/Titles", "Experience Levels", "Company Context",
    "Certifications/Credentials", "Education/Degrees",

    # Channels & Platforms
    "Marketing Channels/Platforms", "Media/Format Types",

    # Business & Industry
    "General Business", "Industries/Verticals", "Geographic/Market Focus",
    "Compliance/Legal", "Methodologies/Frameworks", "Languages",

    # Data Quality
    "Noise/Invalid"
]

CATEGORY_CONTEXT = {
  # Core Marketing (clarifications for ambiguous ones)
  "Creative": "Advertising creative, campaign concepts, creative strategy and direction",
  "Design": "Visual design, UX/UI design, graphic design, web design, design thinking",
  "Strategy": "Business strategy, competitive strategy, strategic planning, market strategy",
  "Content Marketing": "Content marketing, content creation, content strategy, editorial planning",
  "Data & Analytics": "Analytics, reporting, metrics, performance measurement, business intelligence",
  "Brand": "Brand management, brand strategy, brand positioning, brand development",
  "Product Marketing": "Product launch, product positioning, go-to-market strategy, product messaging",
  "Customer Experience": "Customer journey, customer satisfaction, experience design, customer success",
  "Growth Marketing": "Growth strategy, performance marketing, conversion optimization, growth hacking",
  "Marketing Operations": "Process optimization, marketing technology, workflow management, MarTech",

  # Technology & Tools
  "Software/Tools": "Marketing software, design tools, CRM platforms, automation tools (e.g. Photoshop, Salesforce, HubSpot, Excel, Canva)",
  "Technical/Coding": "Programming languages, web development, coding skills (e.g. HTML, CSS, JavaScript, Python, SQL)",
  "Equipment/Hardware": "Physical tools, camera equipment, recording devices, presentation technology",

  # Skills & Competencies
  "Soft Skills": "Interpersonal skills, communication, leadership, teamwork, problem-solving, time management",
  "General Management": "Project management, team management, budget management, resource planning, vendor management",
  "Personal Attributes": "Personality traits, work style descriptors (e.g. creative, analytical, detail-oriented, results-driven)",

  # Professional Context
  "Job Functions/Titles": "Job titles, roles, positions (e.g. manager, director, coordinator, analyst, specialist)",
  "Experience Levels": "Years of experience, seniority indicators (e.g. '5 years experience', 'senior level', 'entry level')",
  "Company Context": "Company type, organizational context (e.g. startup, enterprise, agency, in-house)",
  "Certifications/Credentials": "Professional certifications, credentials, licenses (e.g. Google Analytics certified, PMP, HubSpot certified)",
  "Education/Degrees": "Educational background, degrees, academic qualifications (e.g. MBA, marketing degree, bachelor's)",

  # Channels & Platforms
  "Marketing Channels/Platforms": "Specific platforms, channels, networks (e.g. Facebook, LinkedIn, Google Ads, TikTok, YouTube)",
  "Media/Format Types": "Content formats, media types (e.g. brochures, flyers, banners, podcasts, webinars, videos, presentations)",

  # Business & Industry
  "General Business": "General business skills, finance, operations (e.g. budgeting, forecasting, procurement, P&L management)",
  "Industries/Verticals": "Industry sectors, market verticals (e.g. healthcare, B2B, retail, SaaS, financial services)",
  "Geographic/Market Focus": "Geographic expertise, regional focus (e.g. international, US market, regional, local markets)",
  "Compliance/Legal": "Legal requirements, regulations, compliance (e.g. GDPR, privacy regulations, advertising standards)",
  "Methodologies/Frameworks": "Business methodologies, frameworks (e.g. Agile, Lean, Design Thinking, Scrum, Six Sigma)",
  "Languages": "Language skills, linguistic abilities (e.g. Spanish, French, bilingual, multilingual)",

  # Data Quality
  "Noise/Invalid": "Typos, gibberish, irrelevant entries, duplicates, unclear or invalid data"
}

SYS_TMPL_CATEGORY_CLASSIFICATION = """You are a marketing skills expert. Classify each skill into one of these 52 categories:

{skill_categories}

BOUNDARY FILTERING GUIDELINES:
- Skills with "manager," "director," "coordinator," "specialist," "analyst" → Job Functions/Titles (not Marketing Management)
- Software names (Photoshop, Salesforce, Excel, HubSpot, Canva) → Software/Tools (not their marketing use)
- Programming languages (HTML, CSS, JavaScript, Python, SQL) → Technical/Coding
- Personality traits (creative, analytical, detail-oriented) → Personal Attributes (not Creative)
- Years of experience, seniority levels → Experience Levels
- Communication, leadership, teamwork → Soft Skills
- Degrees, certifications, education → Education/Degrees or Certifications/Credentials
- Platform names (Facebook, LinkedIn, TikTok) → Marketing Channels/Platforms
- Typos, gibberish, unclear text → Noise/Invalid

GUIDELINES:
- Choose the most specific category that best fits the skill
- Focus on primary function and context, not secondary uses
- For software/tools, use "Software/Tools" not marketing categories
- For job titles, use "Job Functions/Titles"
- For soft skills like "communication", use "Soft Skills"
- For typos or gibberish, use "Noise/Invalid" """

USER_TMPL_CATEGORY_CLASSIFICATION = """Classify these marketing skills. Respond with only the category name for each skill. do not add any explanations or additional text or comments. one per line:

SKILLS:
{skills_text}

CATEGORIES (one per line):"""

In [4]:
raw_df = pd.read_csv('marketing manager skills linkedin profile.csv')

client = OpenAI()
model = "gpt-4o-2024-08-06"

skill_categories_lowercase = [category.lower() for category in SKILL_CATEGORIES]
category_context_lowercase = {category.lower(): context for category, context in CATEGORY_CONTEXT.items()}

def merge_profiles(df: pd.DataFrame):
  df_profiles = pd.DataFrame()
  df_profiles["Profile"] = ""

  for idx, row in df.iterrows():
    row_text = "\n".join([f"{column}: {row[column]}" for column in df.columns if not pd.isna(row[column])])
    df_profiles.loc[idx, "Profile"] = row_text
  return df_profiles


def predict(experience_input):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": SYS_TMPL_EXTRACT_SKILLS,
          },
          {
              "role": "user",
              "content": USER_TMPL_EXTRACT_SKILLS.format(experience_input=experience_input),
          }
      ],
      model=model
  )
  return chat_completion.choices[0].message.content.strip()


def predict_categories(skills_text: str):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": SYS_TMPL_CATEGORY_CLASSIFICATION.format(skill_categories=SKILL_CATEGORIES),
          },
          {
              "role": "user",
              "content": USER_TMPL_CATEGORY_CLASSIFICATION.format(skills_text=skills_text),
          }
      ],
      model=model
  )
  return chat_completion.choices[0].message.content.strip()


def extract_skills(idx_row):
    idx, row = idx_row
    try:
        skills = [skill.strip().title() for skill in predict(row['Profile']).split(",")]
        return idx, skills
    except Exception as e:
        print(f"Error at index {idx}: {e}")
        return idx, []


In [10]:
raw_df = raw_df[:10]    # 10 rows for testing
raw_df

,jobTitleGroup,jobTitle,summary,description1,currentCompanyDescription,name,companyName,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,skills_marketing_manager,Senior Marketing Manager,NaN,NaN,NaN,Neal Duffy,Grenade®,NaN,NaN,NaN,NaN,NaN
1,skills_marketing_manager,"Senior Manager, Partnership Marketing",High-performing and strategic thinking profess...,Managing global partnership marketing and acti...,Managing global partnership marketing and acti...,Ara Leo Messerlian,"Cloud9 Esports, Inc.",NaN,NaN,NaN,NaN,NaN
2,skills_marketing_manager,International Marketing Manager,An international marketing executive with over...,As the world’s leading media and events organi...,As the world’s leading media and events organi...,Mattia Miglio,Cosmoprof Worldwide Bologna,NaN,NaN,NaN,NaN,NaN
3,skills_marketing_manager,Marketing Manager,Experienced Marketing Manager with a strong ba...,NaN,NaN,Mariana Pires,New Balance,NaN,NaN,NaN,NaN,NaN
4,skills_marketing_manager,Global Marketing Manager,My passion is to engage and create experiences...,NaN,NaN,Kathryn Dawson-Lee,Windsor Smith Footwear,NaN,NaN,NaN,NaN,NaN
5,skills_marketing_manager,Board Member,You could say that I have a typical T-shaped p...,Thule is a consumer goods company developing p...,Thule is a consumer goods company developing p...,Therese Reuterswärd,Thule Group,NaN,NaN,NaN,NaN,NaN
6,skills_marketing_manager,Marketing & Communication Manager,A strong concern of well balanced and holistic...,Passionnée par l’évolution consciente du conso...,Passionnée par l’évolution consciente du conso...,Anne Allemand,CDH Group - Centaure Duarib Haemmerlin,NaN,NaN,NaN,NaN,NaN
7,skills_marketing_manager,Affiliate Marketing Manager,Experienced affiliate marketing professional w...,Tasked with expanding the affiliate channel wi...,Tasked with expanding the affiliate channel wi...,Laura Holzwasser,Chewy,NaN,NaN,NaN,NaN,NaN
8,skills_marketing_manager,Marketing Manager Nederland,NaN,NaN,NaN,Sjors van Drie,Anheuser-Busch InBev,NaN,NaN,NaN,NaN,NaN
9,skills_marketing_manager,Global E-commerce & digital marketing manager,- Deep expertise (18-year experience) in e-com...,In charge of Global E-commerce / Digital marke...,In charge of Global E-commerce / Digital marke...,Olivier Gosset,Valmont Group,NaN,NaN,NaN,NaN,NaN


In [11]:
df = merge_profiles(raw_df)
df['Skill'] = ""

results = []

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(extract_skills, (idx, row)) for idx, row in df.iterrows()]
    for future in as_completed(futures):
        results.append(future.result())

for idx, skills in results:
    df.at[idx, 'Skill'] = skills

df = df.drop(columns=['Profile']).explode('Skill')
df = df[~df['Skill'].isna()]
df = df[df['Skill'].astype(bool)]
df = df['Skill'].value_counts().reset_index(name='Count')
df.columns = ['Skill', 'Count']

In [12]:
df

,Skill,Count
0,Null,4
1,Marketing,2
2,Seo,2
3,Web Analytics,2
4,Crm,2
...,...,...
108,Technology,1
109,Digital Innovation,1
110,Social Media,1
111,Advertising,1


In [13]:
def process_batch(batch_data, start_idx):
    text_skills = "\n".join(batch_data['Skill'].tolist())
    output = predict_categories(text_skills)
    output_categories = output.split("\n")

    result = []

    if len(output_categories) == len(batch_data):
        for idx, category in enumerate(output_categories):
            skill_index = start_idx + idx
            category = category.strip()
            score = 0.3
            lvl = "Low"

            if category.lower() in skill_categories_lowercase:
                score = 0.75
                lvl = "Medium"

                if category.lower() in category_context_lowercase:
                    score = 0.8
                    lvl = "High"

            result.append((skill_index, category, score, lvl))
    else:
        print(f"[Warning] Mismatch at batch starting at {start_idx}")
        raise Exception("Mismatch at batch starting at {start_idx}")

    return result


batch = 10
batches = [(df.iloc[i:i + batch].copy(), i) for i in range(0, len(df), batch)]

results = []

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(process_batch, batch_data, start_idx) for batch_data, start_idx in batches]

    for future in as_completed(futures):
        try:
            results.extend(future.result())
        except Exception as e:
            print(f"[Error] {e}")

for idx, category, score, lvl in results:
    df.at[idx, 'Category'] = category
    df.at[idx, 'Category_Confidence'] = score
    df.at[idx, 'Category_Confidence_Level'] = lvl

In [14]:
df

,Skill,Count,Category,Category_Confidence,Category_Confidence_Level
0,Null,4,Noise/Invalid,0.80,High
1,Marketing,2,Noise/Invalid,0.80,High
2,Seo,2,SEO,0.75,Medium
3,Web Analytics,2,Data & Analytics,0.80,High
4,Crm,2,CRM,0.75,Medium
...,...,...,...,...,...
108,Technology,1,Technical/Coding,0.80,High
109,Digital Innovation,1,Strategy,0.80,High
110,Social Media,1,Social Media,0.75,Medium
111,Advertising,1,Advertising,0.75,Medium


In [15]:
categories_to_remove = [
    'Noise/Invalid',
    'Languages',
    'Geographic/Market Focus',
    'General Management',
    'General Business',
    'Soft Skills',
    'Software/Tools',
    'Job Functions/Titles',
    'Personal Attributes'
]

excluded_categories = {
  'Software/Tools', 'Personal Attributes', 'Industries/Verticals',
  'Job Functions/Titles', 'Experience Levels', 'Languages',
  'Education/Degrees', 'Certifications/Credentials', 'Equipment/Hardware',
  'Company Context', 'Geographic/Market Focus', 'Noise/Invalid'
}

categories_to_remove = categories_to_remove + list(excluded_categories)

filtered_df = df[~df['Category'].isin(categories_to_remove)]
filtered_df = filtered_df.drop_duplicates(subset=['Skill', 'Category'])

categories = df['Category'].unique()
category_sizes = filtered_df.groupby('Category').size().to_dict()
categories = sorted(categories, key=lambda x: category_sizes.get(x, 0), reverse=True)

In [16]:
filtered_df

,Skill,Count,Category,Category_Confidence,Category_Confidence_Level
2,Seo,2,SEO,0.75,Medium
3,Web Analytics,2,Data & Analytics,0.80,High
4,Crm,2,CRM,0.75,Medium
5,Content,2,Content Marketing,0.80,High
10,Amazon,1,Marketing Channels/Platforms,0.80,High
...,...,...,...,...,...
108,Technology,1,Technical/Coding,0.80,High
109,Digital Innovation,1,Strategy,0.80,High
110,Social Media,1,Social Media,0.75,Medium
111,Advertising,1,Advertising,0.75,Medium


In [18]:
category_sizes

{'Advertising': 1,
 'Affiliate Marketing': 3,
 'Brand': 3,
 'CRM': 2,
 'Campaign Management': 2,
 'Channel Marketing': 1,
 'Compliance/Legal': 3,
 'Content Marketing': 1,
 'Customer Experience': 3,
 'Data & Analytics': 4,
 'Digital Advertising': 4,
 'Email Marketing': 1,
 'Events': 4,
 'Influencer Marketing': 1,
 'Market Research': 1,
 'Marketing Automation': 1,
 'Marketing Channels/Platforms': 2,
 'Marketing Management': 4,
 'Marketing Operations': 2,
 'Media Management': 1,
 'Media/Format Types': 2,
 'Methodologies/Frameworks': 1,
 'Product Marketing': 2,
 'Public Relations & Communications': 1,
 'SEO': 2,
 'Sales Management': 1,
 'Social Media': 3,
 'Strategy': 7,
 'Technical/Coding': 1,
 'Trade Marketing': 4}

In [17]:
categories

['Strategy',
 'Data & Analytics',
 'Digital Advertising',
 'Events',
 'Marketing Management',
 'Trade Marketing',
 'Affiliate Marketing',
 'Compliance/Legal',
 'Social Media',
 'Customer Experience',
 'Brand',
 'SEO',
 'CRM',
 'Marketing Channels/Platforms',
 'Campaign Management',
 'Marketing Operations',
 'Media/Format Types',
 'Product Marketing',
 'Content Marketing',
 'Marketing Automation',
 'Email Marketing',
 'Sales Management',
 'Channel Marketing',
 'Methodologies/Frameworks',
 'Influencer Marketing',
 'Media Management',
 'Public Relations & Communications',
 'Market Research',
 'Technical/Coding',
 'Advertising',
 'Noise/Invalid',
 'Software/Tools',
 'General Business',
 'Industries/Verticals',
 'Soft Skills',
 'Geographic/Market Focus',
 'Job Functions/Titles',
 'Experience Levels',
 'Personal Attributes']

In [19]:
def create_category_prompt(parent_category: str, skills_data: pd.DataFrame) -> str:
        """Create enhanced prompt with skill examples."""

        # Sort skills by count for better context
        skills_sorted = skills_data.sort_values('Count', ascending=False)

        # Get top skills as examples
        top_skills = skills_sorted.head(10)['Skill'].tolist()
        all_skills = skills_sorted['Skill'].tolist()

        # Format skills with counts for context
        skills_formatted = []
        for _, row in skills_sorted.iterrows():
            skills_formatted.append(f"- {row['Skill']} (count: {row['Count']})")
        skills_text = '\n'.join(skills_formatted)

        prompt = f"""Analyze the {parent_category} category with {len(skills_data)} skills and design child subcategories.

**TOP SKILLS (by frequency):**
{', '.join(top_skills)}

**OBJECTIVE:** Create meaningful child categories that represent distinct specializations within {parent_category}.

**DESIGN PRINCIPLES:**
- Focus on business functions and career specializations
- Group skills by common purpose, workflow stage, or outcome type
- Child names should be clear to recruiters and professionals
- Avoid tool-specific names unless the entire group is about that tool
- Each child should contain at least 5-10 skills ideally
- Generic "{parent_category} General" grouping only for truly generic skills

**ALL SKILLS IN {parent_category.upper()}:**
{skills_text}

**OUTPUT FORMAT:** JSON array of child category objects:
[
  {{
    "child_name": "Professional function name",
    "description": "Brief explanation of this specialization",
    "rationale": "Why this grouping makes business sense",
    "example_skills": ["skill1", "skill2", "skill3", "skill4", "skill5"],
    "estimated_skill_count": approximate_number
  }}
]

Ensure example_skills are actual skills from the list above.
Respond with JSON only."""

        return prompt

In [20]:
def extract_json_array(text: str):
    match = re.search(r'(\[.*\])', text, re.DOTALL)
    if not match:
        return json.loads(text)
    json_str = match.group(1)
    return json.loads(json_str)


def predict_subcategories(parent_category: str, category_data: pd.DataFrame):
  prompt = create_category_prompt(parent_category, category_data)
  chat_completion = client.chat.completions.create(
      model=model,
      messages=[{"role": "user", "content": prompt}]
  )
  return extract_json_array(chat_completion.choices[0].message.content.strip())

In [21]:
filtered_df

,Skill,Count,Category,Category_Confidence,Category_Confidence_Level
2,Seo,2,SEO,0.75,Medium
3,Web Analytics,2,Data & Analytics,0.80,High
4,Crm,2,CRM,0.75,Medium
5,Content,2,Content Marketing,0.80,High
10,Amazon,1,Marketing Channels/Platforms,0.80,High
...,...,...,...,...,...
108,Technology,1,Technical/Coding,0.80,High
109,Digital Innovation,1,Strategy,0.80,High
110,Social Media,1,Social Media,0.75,Medium
111,Advertising,1,Advertising,0.75,Medium


In [ ]:
'''
Category: programmation
Skills list: python, java, c++, javascript, sql, nosql, etc.

Subcategories: AI, Data Science, Web Development, Mobile Development, etc.

Subcategories: Artificial Intelligence, Data Science, Web Development, Mobile Development, etc.
'''

In [22]:
filtered_df['Subcategory'] = ""

def process_category(category):
    parent_category = category
    skills_data = filtered_df[filtered_df['Category'] == parent_category]
    skills_list = skills_data['Skill'].tolist()
    max_rows = 50
    result = []

    if len(skills_list) <= max_rows:
        children = predict_subcategories(parent_category, skills_data)
        result.extend([
            {"sub_category": child['child_name'], "skills": child['example_skills']} for child in children
        ])
    else:
        list_skills_data = [skills_data.iloc[i:i + max_rows].reset_index(drop=True)
                            for i in range(0, len(skills_data), max_rows)]
        for batch_data in list_skills_data:
            children = predict_subcategories(parent_category, batch_data)
            result.extend([
                {"sub_category": child['child_name'], "skills": child['example_skills']} for child in children
            ])

    return result

all_results = []

with ThreadPoolExecutor(max_workers=20) as executor:
    future_to_category = {executor.submit(process_category, category): category for category in categories}
    for future in as_completed(future_to_category):
        try:
            all_results.extend(future.result())
        except Exception as e:
            print(f"Error processing category {future_to_category[future]}: {e}")

for child in all_results:
    filtered_df.loc[filtered_df['Skill'].isin(child['skills']), 'Subcategory'] = child['sub_category']

In [23]:
filtered_df

,Skill,Count,Category,Category_Confidence,Category_Confidence_Level,Subcategory
2,Seo,2,SEO,0.75,Medium,SEO Strategy and Planning
3,Web Analytics,2,Data & Analytics,0.80,High,Web Analytics and Optimization
4,Crm,2,CRM,0.75,Medium,CRM Strategy Development
5,Content,2,Content Marketing,0.80,High,Content Analytics and Optimization
10,Amazon,1,Marketing Channels/Platforms,0.80,High,Digital Retail and E-commerce
...,...,...,...,...,...,...
108,Technology,1,Technical/Coding,0.80,High,
109,Digital Innovation,1,Strategy,0.80,High,Business Planning and Innovation
110,Social Media,1,Social Media,0.75,Medium,General Social Media Skills
111,Advertising,1,Advertising,0.75,Medium,Digital Advertising


In [ ]:
filtered_df

,Skill,Count,Category,Category_Confidence,Category_Confidence_Level,Subcategory
1,Marketing,2,Marketing Management,0.75,Medium,Digital Marketing Strategy
2,Content,2,Content Marketing,0.80,High,Content Creation and Production
3,Social Media,2,Social Media,0.75,Medium,Social Media Operations and Management
4,Web Analytics,2,Data & Analytics,0.80,High,Web and E-commerce Analytics
6,Seo,2,SEO,0.75,Medium,SEO Strategy and Planning
...,...,...,...,...,...,...
101,Analytics,1,Data & Analytics,0.80,High,Data Analysis and Interpretation
102,Dashboarding,1,Data & Analytics,0.80,High,Visualization and Reporting
103,Datamining,1,Data & Analytics,0.80,High,Advanced Data Techniques
104,Tag Management,1,Data & Analytics,0.80,High,Web and E-commerce Analytics


In [24]:
all_results

[{'sub_category': 'CRM Strategy Development', 'skills': ['Crm', 'E-Crm']},
 {'sub_category': 'Product Strategy and Planning',
  'skills': ['Product Integration', 'Product Positioning']},
 {'sub_category': 'Promotion Strategy Development',
  'skills': ['Promotions', 'Consumer Promotions']},
 {'sub_category': 'Collaborative Marketing Initiatives',
  'skills': ['Partnership Marketing', 'Strategic Partnerships']},
 {'sub_category': 'Strategic Campaign Planning',
  'skills': ['Campaign Planning']},
 {'sub_category': 'Sponsorship and Asset Management',
  'skills': ['Sponsorship Asset Execution']},
 {'sub_category': 'SEO Strategy and Planning', 'skills': ['Seo']},
 {'sub_category': 'Search Engine Marketing (SEM)', 'skills': ['Sem']},
 {'sub_category': 'Information Security Compliance',
  'skills': ['Security Management']},
 {'sub_category': 'Data Protection and Privacy Compliance',
  'skills': ['Data Privacy', 'Gdpr']},
 {'sub_category': 'Field Marketing Strategy',
  'skills': ['Field Marketi

In [18]:
filtered_df.to_csv("example_output_10_rows.csv")